# 2.6. 用RAGAS评测答疑机器人

## 🚄 前言

新人答疑机器人在实际使用中可能会有一些问题，比如提供不准确的答案，答非所问。

例如，当新人提问“如何请假”时，机器人可能给出通用的回答，而不是基于制度文件内容进行回答。你意识到需要对RAG应用进行优化。

优化前，你首先需要找到优化的方向是哪里，即RAG应用还有哪些地方做得不够好。是没有召回合适的文本段，还是在召回正确文本段基础上生成了不正确的内容？也许五条左右的数据你还可以通过人工排查的方法确定问题，但是当数据量变大之后，人工方法就变得困难。

Ragas可以从多个维度对你的RAG应用进行评测。
你只需要准备好问题（question）、RAG应用生成的答案（answer）、RAG应用召回的文本段（contexts）以及问题对应的正确答案（ground_truth），便可以通过大模型和Embedding 模型自动化地、多维度地对RAG应用进行打分。在获取到分数之后，你就可以更有针对性地从RAG应用得分较低的维度进行优化了。

## 🍁 课程目标
学完本课程后，你将能够：<br>
- 了解RAGAS常见指标的含义； 
- 掌握通过RAGAS进行RAG应用评测的方法。

<div margin="40px">
<img src="https://img.alicdn.com/imgextra/i4/O1CN01b2lVQp21JZCJy6Nfe_!!6000000006964-0-tps-739-420.jpg" width="500">
</div>

## 📖 课程目录

- [1. 计算环境准备](#💻-1-计算环境准备)
- [2. 评测数据准备](#🧬-2-评测数据准备)
- [3. 导入百炼大模型与embedding模型](#🤖-3-导入百炼大模型与embedding模型)
- [4. 各评测指标介绍](#📊-4-各评测指标介绍)
    - [4.1. answer_relevancy](#41-answer_relevancy)
    - [4.2. faithfulness](#42-faithfulness)
    - [4.3. context_recall](#43-context_recall)
    - [4.4. context_precision](#44-context_precision)
    - [4.5. answer_correctness](#45-answer_correctness)
- [5. 使用多个指标同时评测](#📝-5-使用多个指标同时评测)


## 💻 1. 计算环境准备

### 🔧 安装依赖

In [ ]:
! pip install -r requirements.txt

### ➕ 导入依赖

In [1]:
import os
import getpass
from langchain_community.llms.tongyi import Tongyi
from langchain_community.embeddings import DashScopeEmbeddings
from datasets import Dataset 
from ragas import evaluate
from ragas.metrics import faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
import pandas as pd

/Users/liupeter/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### ⚙️ 设置环境变量

In [ ]:
import sys
sys.path.append("../")
from config.load_key import load_key
load_key()
print(os.environ["DASHSCOPE_API_KEY"])

## 🧬 2. 评测数据准备
要进行RAGAS评测，你需要准备四种数据，分别是：
1. 问题（question）
2. RAG应用生成的答案（answer）
3. RAG应用召回的文本段（contexts）
4. 问题对应的正确答案（ground_truth）

其中：question与answer都比较容易获得，ground_truth可以通过人工或性能强大的大模型达标；如果RAG应用是你自己搭建的，那么获取contexts也比较容易。

如果你使用的是百炼控制台创建的[RAG应用](https://bailian.console.aliyun.com/#/app-center)，可以参考以下代码获取contexts：

> 注意：你需要通过百炼创建应用，并替换如下的 ```app_id='xxx'```

In [ ]:
from dashscope import Application

response = Application.call(
    app_id='xxx',
    prompt='有什么手机',
    # 在调用时设置has_thoughts为True，就可以在返回找到contexts内容
    has_thoughts = True
    )
response.output.thoughts

## 🤖 3. 导入百炼大模型与embedding模型

ragas默认使用的是openai的llm和embedding模型，我们更换为qwen系列模型。现在qwen模型性能比较强劲的有qwen-max-latest与qwen-plus-latest。

### 3.1. 大模型

你可以用langchain封装好的Tongyi类直接实例化一个llm对象。

In [6]:
llm = Tongyi(model_name="qwen-plus-latest")
llm.invoke("你好")

'你好！有什么可以帮助你的吗？'

也可以用自定义的API创建llm，这种方法的优势是可以自由定制，比如可以看到输入与输出的中间结果，在ragas评测时可以溯源打出分数的原因。
自定义创建llm方法请参考utils文件夹中的custom_llm.py。

In [ ]:
from utils.custom_llm import CustomLLM
llm = CustomLLM()
llm.invoke("你好")

### 3.2. 直接用langchain封装好的DashScopeEmbeddings类

In [ ]:
embedding = DashScopeEmbeddings(model="text-embedding-v3")
embedding.embed_query("你好")[:10]

##  📊 4. 各评测指标介绍

### 4.1. answer_relevancy

#### 概念与计算方法

该指标用于评估answer与给定question的相关性。需要传入question、answer、contexts。评测需要使用大模型与embedding模型。

相关性的通俗含义为：answer是否是对question的回应，如果答非所问，那么相关性会降低。

计算过程如下：

1.  如果大模型判断answer是“不置可否”的，则answer_relevancy直接为0。“不置可否”的含义是：回避、模糊或模棱两可的答案。例如，“我不知道”或“我不确定”。如果answer不是“不置可否”的，则进行以下运算：
    
2.  由大模型对answer反向生成n（默认为3）个问题gen_query<sub>i</sub> ，通过embedding模型，计算gen_query<sub>i</sub> 与原始问题的相似度，并求均值。这个均值就是answer_relevancy。

#### 实现代码

In [ ]:
data_samples = {
    "question": ["第一届超级碗是什么时候？", "赢得超级碗最多的球队是谁？"],
    "answer": ["第一届超级碗于1967年1月15日举行", "我也不知道，但可能是新英格兰爱国者。"],
    "contexts": [
        ["第一届 AFL–NFL 世界冠军赛是一场美式足球比赛，于1967年1月15日在洛杉矶的洛杉矶纪念体育馆举行。"],
        ["新英格兰爱国者和匹兹堡钢人并列最多。", "爱国者和钢人均赢得6次超级碗。"]
    ]
}

dataset = Dataset.from_dict(data_samples)
score = evaluate(
    dataset = dataset,
    metrics=[answer_relevancy],
    llm=llm,
    embeddings=embedding
)
score.to_pandas()

我们可以看到，提示词里有大量的英文，这对于中文问答场景是不友好的。我们可以进行一些更改，将prompt适配到中文。

#### 适配到新的prompt

> 中文提示词请前往ragas_prompt文件夹下的chinese_prompt进行查看。

In [ ]:
from ragas_prompt.chinese_prompt import AnswerRelavency
answer_relevancy.question_generation.instruction = AnswerRelavency.question_generation_prompt["instruction"]
answer_relevancy.question_generation.output_format_instruction = AnswerRelavency.question_generation_prompt["output_format_instruction"]
answer_relevancy.question_generation.examples = AnswerRelavency.question_generation_prompt["examples"]
answer_relevancy

适配后进行ragas评测

In [ ]:
data_samples = {
    "question": ["第一届超级碗是什么时候？", "赢得超级碗最多的球队是谁？"],
    "answer": ["第一届超级碗于1967年1月15日举行", "我也不知道，但可能是新英格兰爱国者。"],
    "contexts": [
        ["第一届 AFL–NFL 世界冠军赛是一场美式足球比赛，于1967年1月15日在洛杉矶的洛杉矶纪念体育馆举行。"],
        ["新英格兰爱国者和匹兹堡钢人并列最多。", "爱国者和钢人均赢得6次超级碗。"]
    ]
}

dataset = Dataset.from_dict(data_samples)
score = evaluate(
    dataset = dataset,
    metrics=[answer_relevancy],
    llm=llm,
    embeddings=embedding
)
score.to_pandas()

我们可以看到，第一个数据answer_relevancy接近1，第二个数据answer_relevancy为0。

我们通过人工查看一下，这个分数是否合理：

第一届超级碗于1967年1月15日举行 这个答案是对问题：第一届超级碗是什么时候？的回应，因此分数较高；

我也不知道，但可能是新英格兰爱国者。 这个答案，符合“不置可否”的定义，因此分数为0。

### 4.2. faithfulness

#### 概念与计算方法
该指标衡量了answer与contexts的**事实一致性**。需要question、answer、contexts。评测需要使用到大模型。

事实一致性的通俗含义为：RAG应用生成的答案是否遵从召回文本段的内容。如果召回文本段的内容是：“小明喜欢吃米饭”，但是RAG生成的答案为：“小明不喜欢吃米饭”，那么这个答案的faithfulness值会降低。

计算过程如下：

1.  由大模型将answer分解成n个陈述statements<sub>i</sub>。比如answer为：“小明不喜欢吃米饭，喜欢吃面条。”那么大模型可能分解的第一个陈述为：“小明不喜欢吃米饭。”；可能分解的第一个陈述为：“小明喜欢吃面条。”
    
2.  由大模型判断每个statements<sub>i</sub> 能否被contexts推断，并计算能被推断的statements<sub>i</sub> 占总陈述个数的比例，作为faithfulness分数

> 由于ragas的开发人员多使用英语语种，因此为了适配到中文语境，需要做一些调整。找到ragas源码：ragas/metrics/_faithfulness.py中寻找：statements_str: str = json.dumps(statements)，将其改为：statements_str: str = json.dumps(statements,ensure_ascii=False)


#### 实现代码


In [ ]:
data_samples = {
    "question": ["第一届超级碗是什么时候？", "赢得超级碗最多的球队是谁？"],
    "answer": ["第一届超级碗于1967年1月15日举行", "赢得超级碗最多的球队是新英格兰爱国者"],
    "contexts": [
        ["第一届 AFL–NFL 世界冠军赛是一场美式足球比赛，于1967年1月15日在洛杉矶的洛杉矶纪念体育馆举行。"],
        ["绿湾包装工队...威斯康星州绿湾。", "包装工队参加...足球联盟"]
    ]
}

dataset = Dataset.from_dict(data_samples)
score = evaluate(
    dataset = dataset,
    metrics=[faithfulness],
    llm=llm)
score.to_pandas()

#### 适配到新的prompt

In [ ]:
from ragas_prompt.chinese_prompt import Faithfulness

faithfulness.nli_statements_message.instruction = Faithfulness.nli_statements_message_prompt["instruction"]
faithfulness.nli_statements_message.output_format_instruction = Faithfulness.nli_statements_message_prompt["output_format_instruction"]
faithfulness.nli_statements_message.examples = Faithfulness.nli_statements_message_prompt["examples"]
faithfulness.statement_prompt.instruction = Faithfulness.statement_prompt["instruction"]
faithfulness.statement_prompt.output_format_instruction = Faithfulness.statement_prompt["output_format_instruction"]
faithfulness.statement_prompt.examples = Faithfulness.statement_prompt["examples"]
faithfulness

In [ ]:
data_samples = {
    'question': ['第一届超级碗是什么时候？', '谁赢得了最多的超级碗？'],
    'answer': ['第一届超级碗于1967年1月15日举行', '新英格兰爱国者队赢得了最多的超级碗'],
    'contexts' : [['第一届 AFL–NFL 世界锦标赛是一场美式足球比赛，于1967年1月15日在洛杉矶纪念体育场举行，'],
    ['绿湾包装工队...位于威斯康星州绿湾市。', '包装工队参加...足球联盟']],
}

dataset = Dataset.from_dict(data_samples)
score = evaluate(
    dataset = dataset,
    metrics=[faithfulness],
    llm=llm)
score.to_pandas()

我们可以看到，第一个数据faithfulness为1，第二个数据faithfulness为0。

我们通过人工查看一下，这个分数是否合理：

1. 第一届超级碗于1967年1月15日举行 这个答案可以从“第一届 AFL–NFL 世界锦标赛是一场美式足球比赛，于1967年1月15日在洛杉矶纪念体育场举行，”中获取，因此faithfulness分数判定为1；
2. 新英格兰爱国者队赢得了最多的超级碗 这个答案，无法在contexts中找到对应资料，因此faithfulness分数判定为0。
> 你也可以从打印的日志中查看reason字段，了解大模型打分的依据。

### 4.3. context_recall

#### 概念与计算方法

该指标衡量contexts与ground_truth的一致程度。需要传入question、answer、contexts、ground_truth。评测需要大模型。

该一致程度的含义为：召回的文本段是否能够支撑ground_truth，评估的是检索阶段的效果。

计算过程：

1.  由大模型将ground_truth分解成n个陈述statements<sub>i</sub>；
    
2.  由大模型判断每个statements<sub>i</sub> 能否被contexts推断，并计算能被推断的statements<sub>i</sub> 占总陈述个数的比例，作为context_recall分数。


#### 实现代码

In [ ]:
data_samples = {
    'question': ['第一届超级碗是什么时候？', '谁赢得了最多的超级碗？'],
    'answer': ['第一届超级碗于1967年1月15日举行', '新英格兰爱国者队赢得了最多的超级碗'],
    'contexts' : [['第一届 AFL–NFL 世界锦标赛是一场美式足球比赛，于1967年1月15日在洛杉矶纪念体育场举行，'],
    ['绿湾包装工队...位于威斯康星州绿湾市。', '包装工队参加...足球联盟']],
    'ground_truth':['第一届超级碗是在1967年1月15日举行的。','截至2023年，匹兹堡钢人队（Pittsburgh Steelers）和新英格兰爱国者队（New England Patriots）各赢得了六次超级碗冠军，是赢得超级碗次数最多的两支球队。']
}

dataset = Dataset.from_dict(data_samples)
score = evaluate(
    dataset = dataset,
    metrics=[context_recall],
    llm=llm)
score.to_pandas()

#### 适配到新的prompt

In [ ]:
from ragas_prompt.chinese_prompt import ContextRecall
context_recall.context_recall_prompt.instruction = ContextRecall.context_recall_prompt["instruction"]
context_recall.context_recall_prompt.output_format_instruction = ContextRecall.context_recall_prompt["output_format_instruction"]
context_recall.context_recall_prompt.examples = ContextRecall.context_recall_prompt["examples"]
context_recall

In [ ]:
data_samples = {
    'question': ['第一届超级碗是什么时候？', '谁赢得了最多的超级碗？'],
    'answer': ['第一届超级碗于1967年1月15日举行', '新英格兰爱国者队赢得了最多的超级碗'],
    'contexts' : [['第一届 AFL–NFL 世界锦标赛是一场美式足球比赛，于1967年1月15日在洛杉矶纪念体育场举行，'],
    ['绿湾包装工队...位于威斯康星州绿湾市。', '包装工队参加...足球联盟']],
    'ground_truth':['第一届超级碗是在1967年1月15日举行的。','截至2023年，匹兹堡钢人队（Pittsburgh Steelers）和新英格兰爱国者队（New England Patriots）各赢得了六次超级碗冠军，是赢得超级碗次数最多的两支球队。']
}

dataset = Dataset.from_dict(data_samples)
score = evaluate(
    dataset = dataset,
    metrics=[context_recall],
    llm=llm)
score.to_pandas()

我们可以看到，第一个数据context_recall为1，第二个数据context_recall为0。

我们通过人工查看一下，这个分数是否合理：

“第一届 AFL–NFL 世界锦标赛是一场美式足球比赛，于1967年1月15日在洛杉矶纪念体育场举行，”这个context可以支撑“第一届超级碗是在1967年1月15日举行的。”这个ground_truth，因此分数为1；

“绿湾包装工队...位于威斯康星州绿湾市。', '包装工队参加...足球联盟”这个context无法支撑“截至2023年，匹兹堡钢人队（Pittsburgh Steelers）和新英格兰爱国者队（New England Patriots）各赢得了六次超级碗冠军，是赢得超级碗次数最多的两支球队。”这个ground_truth，因此分数为0。

你也可以从打印的日志中查看reason字段，了解大模型打分的依据。

### 4.4. context_precision

#### 概念与计算方法

该指标用于评估contexts中存在的所有真实相关项是否排名较高。需要传入question、contexts、ground_truth，评测需要大模型。

大模型在生成时可能会更关注排名靠前的文本段。该指标的通俗含义为：在召回的所有contexts中，与ground_truth相关的context是否排名较高。

计算过程：

1.  按顺序读取contexts中的context<sub>i</sub> ，根据question与ground_truth，判断该context<sub>i</sub> 是否相关。如果相关，则该项对应的相关分数relative_score<sub>i</sub>为1，否则为0。
2.  计算每一个context<sub>i</sub> 的分数：分子为前i项relative_score<sub>i</sub>的和，分母为context<sub>i</sub>所处的排位。
3.  将所有context<sub>i</sub> 所得分数相加，作为context_precision分数的分子；分母为相关的context<sub>i</sub> 个数。
    
<!-- <img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/ybEnBBXZ6LoPnP13/img/be18e672-edc6-44f6-8136-737f1c057173.png" alt="Alt text" width="500"/> -->

<!-- ![image](https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/ybEnBBXZ6LoPnP13/img/be18e672-edc6-44f6-8136-737f1c057173.png) -->

计算举例，假设已完成计算过程的第一步：contexts列表中以1代表相关，0代表不相关。

*   [1,0,0] 计算方法为：[（1/1）+（0/2）+（0/3）]/ 1 = 1
    
*   [0,1,1] 计算方法为：[（0/1）+（1/2）+（2/3）]/ 2 = 7/12

我们可以看到，第一个例子中，尽管3个contexts中只有一个context是相关的，但是分数为1，因为它把相关的context放在了第一位；第二个例子中，尽管3个contexts中有两个context是相关的，但是分数依然低于第一个例子中的分数，因为它把不相关的context放在了第一位。

#### 实现代码

In [ ]:
data_samples = {
    'question': ['第一届超级碗是什么时候？', '谁赢得了最多的超级碗？'],
    'contexts' : [['爱因斯坦于1895年搬到了瑞士。','第一届 AFL–NFL 世界锦标赛是一场美式足球比赛，于1967年1月15日在洛杉矶纪念体育场举行，'],
    ['匹兹堡钢人队（Pittsburgh Steelers）和新英格兰爱国者队（New England Patriots）赢得最多超级碗。', '匹兹堡钢人队（Pittsburgh Steelers）和新英格兰爱国者队（New England Patriots）各赢得了六次超级碗冠军']],
    'ground_truth':['第一届超级碗是在1967年1月15日举行的。','截至2023年，匹兹堡钢人队（Pittsburgh Steelers）和新英格兰爱国者队（New England Patriots）各赢得了六次超级碗冠军，是赢得超级碗次数最多的两支球队。']
}

dataset = Dataset.from_dict(data_samples)
score = evaluate(
    dataset = dataset,
    metrics=[context_precision],
    llm=llm)
score.to_pandas()

#### 适配到新的prompt

In [ ]:
from ragas_prompt.chinese_prompt import ContextPrecision
context_precision.context_precision_prompt.instruction = ContextPrecision.context_precision_prompt["instruction"]
context_precision.context_precision_prompt.output_format_instruction = ContextPrecision.context_precision_prompt["output_format_instruction"]
context_precision.context_precision_prompt.examples = ContextPrecision.context_precision_prompt["examples"]
context_precision

In [ ]:
data_samples = {
    'question': ['第一届超级碗是什么时候？', '谁赢得了最多的超级碗？'],
    'answer': ['第一届超级碗于1967年1月15日举行', '新英格兰爱国者队赢得了最多的超级碗'],
    'contexts' : [['爱因斯坦于1895年搬到了瑞士。','第一届 AFL–NFL 世界锦标赛是一场美式足球比赛，于1967年1月15日在洛杉矶纪念体育场举行，'],
    ['匹兹堡钢人队（Pittsburgh Steelers）和新英格兰爱国者队（New England Patriots）赢得最多超级碗。', '匹兹堡钢人队（Pittsburgh Steelers）和新英格兰爱国者队（New England Patriots）各赢得了六次超级碗冠军']],
    'ground_truth':['第一届超级碗是在1967年1月15日举行的。','截至2023年，匹兹堡钢人队（Pittsburgh Steelers）和新英格兰爱国者队（New England Patriots）各赢得了六次超级碗冠军，是赢得超级碗次数最多的两支球队。']
}

dataset = Dataset.from_dict(data_samples)
score = evaluate(
    dataset = dataset,
    metrics=[context_precision],
    llm=llm)
score.to_pandas()

我们可以看到，第一个数据context_precision为0.5，第二个数据context_precision为1。

我们通过人工查看一下，这个分数是否合理：

1. 第一个context是讲述爱因斯坦的生平，这与question与ground_truth无关；第二个context有关。计算[0,1]的分数为：[(0/1)+(1/2)]/2 = 1/2
2. 这两个context都与question与ground_truth相关。计算[1,1]的分数为：[(1/1)+(2/2)]/2 = 1

这与运行程序打分的结果一致。

你也可以从打印的日志中查看reason字段，了解大模型打分的依据。

### 4.5. answer_correctness

#### 概念与计算方法

该指标评估回答的准确性。包括事实准确性（f1 score）与语义相似度（similarity score）。需要传入question、answer、ground_truth，需要大模型与embedding模型。

该分数可作为端到端的指标，评估RAG应用的整体效果。

<!-- 评估回答的准确性。包括事实准确性（f1 score）与语义相似度（similarity score），加权相加，默认是3:1。语义相似度用embedding模型计算，比较二者向量的余弦相似度。需要answer、ground_truth -->
事实准确性（使用f1 score）计算过程：

1.  通过大模型将answer、ground_truth分别生成各自的陈述列表。比如：  
    **answer**:爱因斯坦于 1879 年出生于西班牙 ---> ["爱因斯坦1879年出生","爱因斯坦出生于西班牙"]  
    **ground_truth**：爱因斯坦 1879 年出生于德国。---> ["爱因斯坦1879年出生","爱因斯坦出生于德国"]
    
2.  遍历answer与ground_truth列表，并初始化三个列表，TP、FP与FN；  
    对于**answer**列表中的陈述来说：如果该陈述**可以**被ground_truth支持，则将该陈述添加到TP列表中；如果该陈述**不可以**被ground_truth支持，则将该陈述添加到FP列表中；  
    对于**ground_truth**列表中的陈述来说：如果该陈述**不可以**被answer支持，则将该陈述添加到FN列表中；  
    举例：TP：\["爱因斯坦1879年出生"\]，FP：\["爱因斯坦出生于西班牙"\]，FN：\["爱因斯坦出生于德国"\]  
    <!-- ![image](https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/ybEnBBXZ6LoPnP13/img/86c4839b-bd48-4faf-9842-c42599181339.png) -->
    
3.  统计TP、FP与FN列表的元素个数，并按照以下方式计算f1 score分数：
    

```shell
f1 score = tp / (tp + 0.5 * (fp + fn)) if tp > 0 else 0
```

以上文为例：f1 score = 1/(1+0.5\*(1+1)) = 0.5

得到了f1 score，以及通过embedding模型计算的answer与ground_truth的相似度（similarity），使用以下方式计算answer_correctness：

$answercorrectness=w1*f1 score+w2*similarity$

其中w1和w2相加为1，默认为0.75和0.25

#### 实现代码

In [ ]:
data_samples = {
    'question': ['第一届超级碗是什么时候？', '谁赢得了最多的超级碗？'],
    'answer': ['第一届超级碗于1967年1月15日举行', '新英格兰爱国者队赢得了最多的超级碗'],
    'ground_truth':['第一届超级碗是在1967年1月15日举行的。','截至2023年，匹兹堡钢人队（Pittsburgh Steelers）和新英格兰爱国者队（New England Patriots）各赢得了六次超级碗冠军，是赢得超级碗次数最多的两支球队。']
}

dataset = Dataset.from_dict(data_samples)
score = evaluate(
    dataset = dataset,
    metrics=[answer_correctness],
    llm=llm,
    embeddings=embedding)
score.to_pandas()

answer_correctness指标的计算用到了faithfulness的prompt，因此有部分prompt是之前适配过的中文。

#### 适配到新的prompt

In [ ]:
from ragas_prompt.chinese_prompt import AnswerCorrectness
answer_correctness.correctness_prompt.instruction = AnswerCorrectness.correctness_prompt["instruction"]
answer_correctness.correctness_prompt.output_format_instruction = AnswerCorrectness.correctness_prompt["output_format_instruction"]
answer_correctness.correctness_prompt.examples = AnswerCorrectness.correctness_prompt["examples"]
answer_correctness

In [ ]:
data_samples = {
    'question': ['第一届超级碗是什么时候？', '谁赢得了最多的超级碗？'],
    'answer': ['第一届超级碗于1967年1月15日举行', '新英格兰爱国者队赢得了最多的超级碗'],
    'ground_truth':['第一届超级碗是在1967年1月15日举行的。','截至2023年，匹兹堡钢人队（Pittsburgh Steelers）和新英格兰爱国者队（New England Patriots）各赢得了六次超级碗冠军，是赢得超级碗次数最多的两支球队。']
}

dataset = Dataset.from_dict(data_samples)
score = evaluate(
    dataset = dataset,
    metrics=[answer_correctness],
    llm=llm,
    embeddings=embedding)
score.to_pandas()

我们可以看到，第一个数据answer_correctness接近1，第二个数据answer_correctness为0.57。

我们通过人工查看一下，这个分数是否合理：

1. answer与ground_truth几乎一样，因此answer_correctness分数接近于1；
2. answer中只提到了“新英格兰爱国者队”，却没有提到”匹兹堡钢人队“，因此分数比前者低。

你也可以从打印的日志中查看reason字段，了解大模型打分的依据。

## 📝 5. 使用多个指标同时评测
### 5.1. 数据准备
你需要准备四种数据，每种数据的类型为：<br>
| question | answer | contexts | ground_truth |
| --- | --- | --- | --- |
| str | str | List[str] | str |


在evaluate方法中，指定metrics参数为你想要评测的指标。

In [ ]:
data_samples = {
    'question': ['第一届超级碗是什么时候？', '谁赢得了最多的超级碗？'],
    'answer': ['第一届超级碗于1967年1月15日举行', '新英格兰爱国者队赢得了最多的超级碗'],
    'contexts' : [['第一届 AFL–NFL 世界锦标赛是一场美式足球比赛，于1967年1月15日在洛杉矶纪念体育场举行，'],
    ['绿湾包装工队...位于威斯康星州绿湾市。', '包装工队参加...足球联盟']],
    'ground_truth':['第一届超级碗是在1967年1月15日举行的。','截至2023年，匹兹堡钢人队（Pittsburgh Steelers）和新英格兰爱国者队（New England Patriots）各赢得了六次超级碗冠军，是赢得超级碗次数最多的两支球队。']
}

dataset = Dataset.from_dict(data_samples)
score = evaluate(
    dataset = dataset,
    metrics=[answer_correctness,answer_relevancy, context_recall,context_precision,faithfulness],
    embeddings=embedding,
    llm=llm)
score.to_pandas()

### 5.2. 函数的封装

我们封装好了一个适配到中文prompt的函数，你只需要传入question、answer、contexts、ground_truth，就可以获得ragas的评测结果。

In [ ]:
data_samples = {
    'question': ['第一届超级碗是什么时候？', '谁赢得了最多的超级碗？'],
    'answer': ['第一届超级碗于1967年1月15日举行', '新英格兰爱国者队赢得了最多的超级碗'],
    'contexts' : [['第一届 AFL–NFL 世界锦标赛是一场美式足球比赛，于1967年1月15日在洛杉矶纪念体育场举行，'],
    ['绿湾包装工队...位于威斯康星州绿湾市。', '包装工队参加...足球联盟']],
    'ground_truth':['第一届超级碗是在1967年1月15日举行的。','截至2023年，匹兹堡钢人队（Pittsburgh Steelers）和新英格兰爱国者队（New England Patriots）各赢得了六次超级碗冠军，是赢得超级碗次数最多的两支球队。']
}

In [ ]:
from utils.ragas_evaluate import ragas_evaluate
ragas_evaluate(
    question=data_samples["question"],
    answer=data_samples["answer"],
    contexts=data_samples["contexts"],
    ground_truth=data_samples["ground_truth"]
)

## ✅ 本节小结
通过本节内容的学习，你已经学会了怎么使用RAGAS框架来评估RAG应用的质量了。